# **Part 2: Product matching**
**Attached is sample product lists from Flipkart & Amazon**

**<DS - Assignment Part 2 data set.zip>**

**Link**: https://www.dropbox.com/sh/aypq6h3254207bs/AACzMLvo-XtK9sYAAma6FW0la?dl=0

**Problem statement:**

Using ML/DL techniques, match similar products from the Flipkart dataset with the Amazon dataset. Once
similar products are matched, display the retail price from FK and AMZ side by side. Please explore as
many techniques as possible before choosing the final technique.
You may either display the final result in single table format OR You may create a simple form where we
input the product name and the output of prices of the product from both websites are displayed.


# **Data Familirization**

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import warnings
warnings.filterwarnings("ignore")

In [2]:
am_df = pd.read_csv(r'amz_com-ecommerce_sample.csv', encoding='latin1')
am_df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [3]:
fk_df = pd.read_csv(r'flipkart_com-ecommerce_sample.csv', encoding='latin1')
fk_df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [4]:
print('-'*50)
print('\033[1m','Shape of the Amazon dataset: {}'. format(am_df.shape),'\033[0m')
print('\033[1m','Shape of the Flipkart dataset: {}'. format(fk_df.shape),'\033[0m')
print('-'*50)

--------------------------------------------------
 Shape of the Amazon dataset: (20000, 15) 
 Shape of the Flipkart dataset: (20000, 15) 
--------------------------------------------------


In [5]:
am_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   uniq_id                  20000 non-null  object
 1   crawl_timestamp          20000 non-null  object
 2   product_url              20000 non-null  object
 3   product_name             20000 non-null  object
 4   product_category_tree    20000 non-null  object
 5   pid                      20000 non-null  object
 6   retail_price             20000 non-null  int64 
 7   discounted_price         20000 non-null  int64 
 8   image                    19997 non-null  object
 9   is_FK_Advantage_product  20000 non-null  bool  
 10  description              19998 non-null  object
 11  product_rating           20000 non-null  object
 12  overall_rating           20000 non-null  object
 13  brand                    14136 non-null  object
 14  product_specifications   19986 non-nul

In [6]:
fk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

## Checking for Null Values

In [7]:
am_df.isna().sum()

uniq_id                       0
crawl_timestamp               0
product_url                   0
product_name                  0
product_category_tree         0
pid                           0
retail_price                  0
discounted_price              0
image                         3
is_FK_Advantage_product       0
description                   2
product_rating                0
overall_rating                0
brand                      5864
product_specifications       14
dtype: int64

In [8]:
fk_df.isna().sum()

uniq_id                       0
crawl_timestamp               0
product_url                   0
product_name                  0
product_category_tree         0
pid                           0
retail_price                 78
discounted_price             78
image                         3
is_FK_Advantage_product       0
description                   2
product_rating                0
overall_rating                0
brand                      5864
product_specifications       14
dtype: int64

Since all the features are not important for our project, therefore we will extract features which will be useful for getting our desired outcomes. 

In [9]:
# Extracting features from Amazon datasets
m_am_df = am_df.drop(['uniq_id', 'crawl_timestamp', 'product_url', 'pid', 'image', 'is_FK_Advantage_product', 'product_rating', 
            'overall_rating', 'brand', 'product_specifications'], axis = 1)

In [10]:
m_am_df.head()

,product_name,product_category_tree,retail_price,discounted_price,description
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",982,438,Key Features of Alisha Solid Women's Cycling S...
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",32143,29121,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",991,551,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",694,325,Key Features of Alisha Solid Women's Cycling S...
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",208,258,Specifications of Sicons All Purpose Arnica Do...


In [11]:
# Extracting features from Flipkart datasets
m_fk_df = fk_df.drop(['uniq_id', 'crawl_timestamp', 'product_url', 'pid', 'image', 'is_FK_Advantage_product', 'product_rating', 
            'overall_rating', 'brand', 'product_specifications'], axis = 1)

In [12]:
m_fk_df.head()

,product_name,product_category_tree,retail_price,discounted_price,description
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",999.0,379.0,Key Features of Alisha Solid Women's Cycling S...
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",999.0,499.0,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",699.0,267.0,Key Features of Alisha Solid Women's Cycling S...
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",220.0,210.0,Specifications of Sicons All Purpose Arnica Do...


In [13]:
m_am_df.isna().sum()

product_name             0
product_category_tree    0
retail_price             0
discounted_price         0
description              2
dtype: int64

In [14]:
m_fk_df.isna().sum()

product_name              0
product_category_tree     0
retail_price             78
discounted_price         78
description               2
dtype: int64

We will handle the missing values of our extracted features.

In [15]:
m_am_df[m_am_df['description'].isna() == True]

,product_name,product_category_tree,retail_price,discounted_price,description
553,Ozel Studio Casual Sleeveless Printed Women's Top,"[""Clothing >> Women's Clothing >> Western Wear...",1278,781,NaN
17299,AMORE ABSTRACT CUSHIONS COVER,"[""Home Furnishing >> Cushions, Pillows & Cover...",483,368,NaN


In [16]:
m_fk_df[m_fk_df['description'].isna() == True]

,product_name,product_category_tree,retail_price,discounted_price,description
553,Ozel Studio Casual Sleeveless Printed Women's Top,"[""Clothing >> Women's Clothing >> Western Wear...",1290.0,645.0,NaN
17299,Amore Abstract Cushions Cover,"[""Home Furnishing >> Cushions, Pillows & Cover...",499.0,299.0,NaN


In [17]:
m_am_df['description'] = np.where((m_am_df.description.isnull() == True ), m_am_df['product_name'], m_am_df['description'])
m_fk_df['description'] = np.where((m_fk_df.description.isnull() == True ), m_fk_df['product_name'], m_fk_df['description'])

In [18]:
m_am_df[m_am_df['description'].isna() == True]

,product_name,product_category_tree,retail_price,discounted_price,description


In [19]:
m_fk_df[m_fk_df['description'].isna() == True]

,product_name,product_category_tree,retail_price,discounted_price,description


In [20]:
m_am_df['product_category_tree'] = m_am_df['product_category_tree'].apply(lambda x :  x.replace(' >>',' ').replace('"','').replace('[','').replace(']',''))
m_fk_df['product_category_tree'] = m_fk_df['product_category_tree'].apply(lambda x :  x.replace(' >>',' ').replace('"','').replace('[','').replace(']',''))

In [21]:
# Filling the null values of 'retail_price' and 'discounted_price' with 0
m_fk_df['retail_price'].fillna(0, inplace = True)
m_fk_df['discounted_price'].fillna(0, inplace = True)

In [22]:
m_am_df = m_am_df.rename(columns = {'product_name':'Product Name in Amazon', 'product_category_tree':'category', 
                                    'retail_price':'Retail Price in Amazon', 'discounted_price':'Discounted Price in Amazon'})
                                    
m_fk_df = m_fk_df.rename(columns = {'product_name':'Product Name in Flipkart', 'product_category_tree':'category', 
                                    'retail_price':'Retail Price in Flipkart','discounted_price':'Discounted Price in Flipkart'})

In [23]:
m_am_df.head()

,Product Name in Amazon,category,Retail Price in Amazon,Discounted Price in Amazon,description
0,Alisha Solid Women's Cycling Shorts,"Clothing Women's Clothing Lingerie, Sleep & ...",982,438,Key Features of Alisha Solid Women's Cycling S...
1,FabHomeDecor Fabric Double Sofa Bed,Furniture Living Room Furniture Sofa Beds & ...,32143,29121,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW Bellies,Footwear Women's Footwear Ballerinas AW Bel...,991,551,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha Solid Women's Cycling Shorts,"Clothing Women's Clothing Lingerie, Sleep & ...",694,325,Key Features of Alisha Solid Women's Cycling S...
4,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies Grooming Skin & Coat Care Sham...,208,258,Specifications of Sicons All Purpose Arnica Do...


In [24]:
m_fk_df.head()

,Product Name in Flipkart,category,Retail Price in Flipkart,Discounted Price in Flipkart,description
0,Alisha Solid Women's Cycling Shorts,"Clothing Women's Clothing Lingerie, Sleep & ...",999.0,379.0,Key Features of Alisha Solid Women's Cycling S...
1,FabHomeDecor Fabric Double Sofa Bed,Furniture Living Room Furniture Sofa Beds & ...,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW Bellies,Footwear Women's Footwear Ballerinas AW Bel...,999.0,499.0,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha Solid Women's Cycling Shorts,"Clothing Women's Clothing Lingerie, Sleep & ...",699.0,267.0,Key Features of Alisha Solid Women's Cycling S...
4,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies Grooming Skin & Coat Care Sham...,220.0,210.0,Specifications of Sicons All Purpose Arnica Do...


After Data Cleaning, now it's time to create a similarity recommendation system. We will be using Product Name, Category and the Description of the products to find the similarity.

In [26]:
features_am = ['Product Name in Amazon', 'category','description']
features_fk = ['Product Name in Flipkart', 'category','description']
am_filters = m_am_df[features_am]
fk_filters = m_fk_df[features_fk]

In [27]:
#Cleaning data by removing spaces and making all the words in lowercase.
def clean(x):
        return str.lower(x.replace(" ", ""))

In [28]:
for feature in am_filters:
    am_filters[feature] = am_filters[feature].apply(clean)
    
am_filters.head()

,Product Name in Amazon,category,description
0,alishasolidwomen'scyclingshorts,"clothingwomen'sclothinglingerie,sleep&swimwear...",keyfeaturesofalishasolidwomen'scyclingshortsco...
1,fabhomedecorfabricdoublesofabed,furniturelivingroomfurnituresofabeds&futonsfab...,fabhomedecorfabricdoublesofabed(finishcolor-le...
2,awbellies,footwearwomen'sfootwearballerinasawbellies,keyfeaturesofawbelliessandalswedgesheelcasuals...
3,alishasolidwomen'scyclingshorts,"clothingwomen'sclothinglingerie,sleep&swimwear...",keyfeaturesofalishasolidwomen'scyclingshortsco...
4,siconsallpurposearnicadogshampoo,petsuppliesgroomingskin&coatcareshampoosiconsa...,specificationsofsiconsallpurposearnicadogshamp...


In [29]:
for feature in fk_filters:
    fk_filters[feature] = fk_filters[feature].apply(clean)
    
fk_filters.head()

,Product Name in Flipkart,category,description
0,alishasolidwomen'scyclingshorts,"clothingwomen'sclothinglingerie,sleep&swimwear...",keyfeaturesofalishasolidwomen'scyclingshortsco...
1,fabhomedecorfabricdoublesofabed,furniturelivingroomfurnituresofabeds&futonsfab...,fabhomedecorfabricdoublesofabed(finishcolor-le...
2,awbellies,footwearwomen'sfootwearballerinasawbellies,keyfeaturesofawbelliessandalswedgesheelcasuals...
3,alishasolidwomen'scyclingshorts,"clothingwomen'sclothinglingerie,sleep&swimwear...",keyfeaturesofalishasolidwomen'scyclingshortsco...
4,siconsallpurposearnicadogshampoo,petsuppliesgroomingskin&coatcareshampoosiconsa...,specificationsofsiconsallpurposearnicadogshamp...


In [30]:
# Creating a new column as 'metadata_soup' which will contain all the metadata from the columns that we want to feed to our vectorizer.
am_filters['metadata_soup'] = am_filters['category'] +' '+ am_filters['description']
fk_filters['metadata_soup'] = fk_filters['category'] +' '+ fk_filters['description']

In [31]:
am_filters.head()

,Product Name in Amazon,category,description,metadata_soup
0,alishasolidwomen'scyclingshorts,"clothingwomen'sclothinglingerie,sleep&swimwear...",keyfeaturesofalishasolidwomen'scyclingshortsco...,"clothingwomen'sclothinglingerie,sleep&swimwear..."
1,fabhomedecorfabricdoublesofabed,furniturelivingroomfurnituresofabeds&futonsfab...,fabhomedecorfabricdoublesofabed(finishcolor-le...,furniturelivingroomfurnituresofabeds&futonsfab...
2,awbellies,footwearwomen'sfootwearballerinasawbellies,keyfeaturesofawbelliessandalswedgesheelcasuals...,footwearwomen'sfootwearballerinasawbellies key...
3,alishasolidwomen'scyclingshorts,"clothingwomen'sclothinglingerie,sleep&swimwear...",keyfeaturesofalishasolidwomen'scyclingshortsco...,"clothingwomen'sclothinglingerie,sleep&swimwear..."
4,siconsallpurposearnicadogshampoo,petsuppliesgroomingskin&coatcareshampoosiconsa...,specificationsofsiconsallpurposearnicadogshamp...,petsuppliesgroomingskin&coatcareshampoosiconsa...


In [32]:
fk_filters.head()

,Product Name in Flipkart,category,description,metadata_soup
0,alishasolidwomen'scyclingshorts,"clothingwomen'sclothinglingerie,sleep&swimwear...",keyfeaturesofalishasolidwomen'scyclingshortsco...,"clothingwomen'sclothinglingerie,sleep&swimwear..."
1,fabhomedecorfabricdoublesofabed,furniturelivingroomfurnituresofabeds&futonsfab...,fabhomedecorfabricdoublesofabed(finishcolor-le...,furniturelivingroomfurnituresofabeds&futonsfab...
2,awbellies,footwearwomen'sfootwearballerinasawbellies,keyfeaturesofawbelliessandalswedgesheelcasuals...,footwearwomen'sfootwearballerinasawbellies key...
3,alishasolidwomen'scyclingshorts,"clothingwomen'sclothinglingerie,sleep&swimwear...",keyfeaturesofalishasolidwomen'scyclingshortsco...,"clothingwomen'sclothinglingerie,sleep&swimwear..."
4,siconsallpurposearnicadogshampoo,petsuppliesgroomingskin&coatcareshampoosiconsa...,specificationsofsiconsallpurposearnicadogshamp...,petsuppliesgroomingskin&coatcareshampoosiconsa...


In [33]:
# Fiting our data in TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')

# Creating the required TF-IDF matrix by fitting and transforming the data
am_multi_matrix = tfidf.fit_transform(am_filters['metadata_soup'])
fk_multi_matrix = tfidf.fit_transform(fk_filters['metadata_soup'])

# Shape of matrices
print('-'*70)
print('\033[1m','Shape of the Amazon Multi Matrix dataset: {}'. format(am_multi_matrix.shape), '\033[0m')
print('\033[1m','Shape of the Flipkart Multi Matrix dataset: {}'. format(fk_multi_matrix.shape), '\033[0m')
print('-'*70)

----------------------------------------------------------------------
 Shape of the Amazon Multi Matrix dataset: (20000, 75808) 
 Shape of the Flipkart Multi Matrix dataset: (20000, 75804) 
----------------------------------------------------------------------


In [34]:
# Computing the Cosine Similarity matrix based on the matrices
am_multi_sim = linear_kernel(am_multi_matrix, am_multi_matrix)
fk_multi_sim = linear_kernel(fk_multi_matrix, fk_multi_matrix)

In [35]:
#Constructing a reverse map of indices with the help of Product Names column
am_indices = pd.Series(am_filters.index, index = am_filters['Product Name in Amazon']).drop_duplicates()
fk_indices = pd.Series(fk_filters.index, index = fk_filters['Product Name in Flipkart']).drop_duplicates()

In [36]:
def similarity(Name):
  Name = Name.replace(' ','').lower()
  am_ids = am_indices[Name]
  fk_ids = fk_indices[Name]

  # Getting the pairwsie similarity scores of all contents with the title
  am_sim_scores = list(enumerate(am_multi_sim[am_ids]))
  fk_sim_scores = list(enumerate(fk_multi_sim[fk_ids]))

  # Sorting the contents based on the similarity scores
  am_sim_scores = sorted(am_sim_scores, key = lambda x: x[1], reverse=True)
  fk_sim_scores = sorted(fk_sim_scores, key = lambda x: x[1], reverse=True)

  # Getting the scores of the 5 most similar contents
  am_sim_scores = am_sim_scores[0:5] 
  fk_sim_scores = fk_sim_scores[0:5]

  # Getting the content indices
  am_content_index = [i[0] for i in am_sim_scores]
  fk_content_index = [i[0] for i in fk_sim_scores]
      
  # Features to be shown
  am_features = ['Product Name in Amazon', 'Retail Price in Amazon','Discounted Price in Amazon']
  fk_features = ['Product Name in Flipkart', 'Retail Price in Flipkart','Discounted Price in Flipkart']

  # Return the top 10 most similar movies
  return pd.concat([m_am_df[am_features].iloc[am_content_index].reset_index(drop = True), 
                        m_fk_df[fk_features].iloc[fk_content_index].reset_index(drop = True)], axis = 1)

# **Testing the Function**

In [37]:
similarity('Sicons All Purpose Arnica Dog Shampoo')

,Product Name in Amazon,Retail Price in Amazon,Discounted Price in Amazon,Product Name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart
0,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0
1,DISNEY HMHILB 169-CIN 1 Containers Lunch Box,161,214,DISNEY HMHILB 169-CIN 1 Containers Lunch Box,169.0,169.0
2,SICONS CONDITIONING CONDITONER DOG SHAMPOO,97,119,Sicons Conditioning Conditoner Dog Shampoo,110.0,100.0
3,Sicons All Purpose Tea Tree Dog Shampoo,-11,0,Sicons All Purpose Tea Tree Dog Shampoo,0.0,0.0
4,MARVEL HMRPLB 256-SPM 1 Containers Lunch Box,192,247,MARVEL HMRPLB 256-SPM 1 Containers Lunch Box,199.0,199.0


In [38]:
similarity('AW Bellies')

,Product Name in Amazon,Retail Price in Amazon,Discounted Price in Amazon,Product Name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart
0,AW Bellies,991,551,AW Bellies,999.0,499.0
1,Glitzy Galz Women Heels,1484,1050,Glitzy Galz Women Heels,1499.0,899.0
2,SHOPNETIX STYLISH & COMFORT MEN'S LACE UP MOCA...,1486,501,Shopnetix Stylish & Comfort Men's Lace up Moca...,1499.0,449.0
3,Lee Parke Running Shoes,988,637,Lee Parke Running Shoes,999.0,499.0
4,Lee Parke Walking Shoes,995,1242,Lee Parke Walking Shoes,999.0,999.0


In [39]:
similarity('MARVEL HMRPLB 256-SPM 1 Containers Lunch Box')

,Product Name in Amazon,Retail Price in Amazon,Discounted Price in Amazon,Product Name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart
0,MARVEL HMRPLB 256-SPM 1 Containers Lunch Box,192,247,MARVEL HMRPLB 256-SPM 1 Containers Lunch Box,199.0,199.0
1,DISNEY HMHILB 169-CIN 1 Containers Lunch Box,161,214,DISNEY HMHILB 169-CIN 1 Containers Lunch Box,169.0,169.0
2,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0
3,MARVEL HMRPLB 73171 - AV 1 Containers Lunch Box,283,346,MARVEL HMRPLB 73171 - AV 1 Containers Lunch Box,299.0,299.0
4,Oros Mild Liquid Detergent with Conditioner Co...,197,232,Oros Mild Liquid Detergent with Conditioner Co...,200.0,185.0
